In [2]:
from TT_utilities import Case, NL_METHODS
from scipy.stats import stats
from pathlib import Path
import pandas as pd
import numpy as np
import re

In [3]:
RECORD_DIRS = list(Path("./Data").glob("*p00*"))
CASES = list()
for record_dir in RECORD_DIRS:
    record_name = re.search("p[0-9]{6}", str(record_dir))[0]
    c = Case(record_dir.joinpath(record_name))
    c.process()
    CASES.append(c)

 > PROCESSING CASE p000638 optimal signal is "II" present in 4 of 5 records.
 > < Record 3033264_0005 - Non linear analysis done.
 > < Record 3033264_0007 - Non linear analysis done.
 > < Record 3033264_0008 - Non linear analysis done.
 > < Record 3033264_0009 - Non linear analysis done.
 < PROCESSED CASE p000638 - 4 valid records kept, 1 records dumped.
Function _non_linear_analysis_c execution time: 91.48's
 > PROCESSING CASE p001170 optimal signal is "II" present in 16 of 19 records.
 > X Record 3612801_0001 - Analysis not possible, rr too short.
 > X Record 3612801_0002 - Analysis not possible, rr too short.
 > < Record 3612801_0008 - Non linear analysis done.
 > X Record 3612801_0011 - Analysis not possible, rr too short.
 > X Record 3612801_0013 - Analysis not possible, rr too short.
 > X Record 3612801_0014 - Analysis not possible, rr too short.
 > < Record 3612801_0017 - Non linear analysis done.
 > < Record 3612801_0020 - Non linear analysis done.
 > < Record 3612801_0022 - No

In [4]:
for c in CASES:
    try:
        c.process()
    except ValueError:
        pass

Case already processed, cant do it again.
Case already processed, cant do it again.
Case already processed, cant do it again.
Case already processed, cant do it again.
Case already processed, cant do it again.
Case already processed, cant do it again.
Case already processed, cant do it again.
Case already processed, cant do it again.


In [6]:
# for mean (_m) and variance (_v)
columns = ["case", "record", "cond"]
for m in NL_METHODS:
    columns.extend([m["tag"]+"_m", m["tag"]+"_v"])
print(columns)

CSV_DATA = pd.DataFrame(columns=columns)
CSV_DATA

['case', 'record', 'cond', 'ae_m', 'ae_v', 'se_m', 'se_v', 'hfd_m', 'hfd_v', 'dfa_m', 'dfa_v', 'psd_m', 'psd_v']


,case,record,cond,ae_m,ae_v,se_m,se_v,hfd_m,hfd_v,dfa_m,dfa_v,psd_m,psd_v


In [7]:
for n, c in enumerate(CASES):
    for r in c:
        vals = list()
        for k, v in r.N_LINEAR.items():
            s = stats.describe(v)
            vals.extend([s[2], s[3]])
        row = [c._case_name, r.name, c.pathology] + vals
        CSV_DATA = CSV_DATA.append(pd.Series(data=row, index=columns), ignore_index=True)
CSV_DATA

,case,record,cond,ae_m,ae_v,se_m,se_v,hfd_m,hfd_v,dfa_m,dfa_v,psd_m,psd_v
0,p000638,3033264_0005,atrial_fibrillation,1.114328,0.006364,1.014952,0.009344,2.062579,0.000213,0.357675,0.003759,0.707119,0.004973
1,p000638,3033264_0007,atrial_fibrillation,1.128314,0.005094,1.019000,0.007935,2.062636,0.000320,0.305381,0.004328,0.670582,0.006041
2,p000638,3033264_0008,atrial_fibrillation,1.182164,0.018420,1.065136,0.042754,2.043882,0.000665,0.339816,0.009309,0.736152,0.008183
3,p000638,3033264_0009,atrial_fibrillation,1.288425,0.024384,1.221510,0.057924,2.020016,0.000103,0.502655,0.002594,0.875502,0.001638
4,p001170,3612801_0008,atrial_fibrillation,0.493513,0.039231,0.290480,0.031252,2.104450,0.001487,0.094455,0.002014,0.313816,0.015252
...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,p000154,3794082_0001,myocardial_infarction,0.582774,0.053045,0.473902,0.058073,2.146280,0.005843,0.133404,0.002195,0.410789,0.016720
64,p000154,3794082_0002,myocardial_infarction,0.818121,0.043284,0.758590,0.084491,2.114590,0.002297,0.151744,0.002885,0.506779,0.007265
65,p000154,3794082_0005,myocardial_infarction,0.672129,0.021658,0.575879,0.030423,2.143030,0.002456,0.117214,0.001214,0.403666,0.015805
66,p000154,3794082_0007,myocardial_infarction,0.806108,0.034007,0.768532,0.046890,2.095581,0.000912,0.138230,0.002504,0.546346,0.003729


In [8]:
CSV_DATA.to_csv("worksample_data.csv")